In [1]:
#Instalação
!pip install pymupdf wget

In [42]:
%%bash
url_base="https://download.inep.gov.br/enem/provas_e_gabaritos"


for ano in {2023..2023}; do
    for dia in {1..12}; do
        for caderno in {1..12}; do
            # Usa a variável na URL corretamente
            wget "${url_base}/${ano}_PV_impresso_D${dia}_CD${caderno}.pdf" -P ./provas --no-check-certificate
        done
    done
done

--2025-02-19 21:49:05--  https://download.inep.gov.br/enem/provas_e_gabaritos/2023_PV_impresso_D1_CD1.pdf
Resolvendo download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Conectando-se a download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... conectado.
Não foi possível estabelecer conexão segura (SSL).
--2025-02-19 21:49:05--  https://download.inep.gov.br/enem/provas_e_gabaritos/2023_PV_impresso_D1_CD2.pdf
Resolvendo download.inep.gov.br (download.inep.gov.br)... 200.130.24.15
Conectando-se a download.inep.gov.br (download.inep.gov.br)|200.130.24.15|:443... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 4261148 (4,1M) [application/pdf]
Salvando em: “./provas/2023_PV_impresso_D1_CD2.pdf”

     0K .......... .......... .......... .......... ..........  1%  391K 11s
    50K .......... .......... .......... .......... ..........  2% 1,11M 7s
   100K .......... .......... .......... .......... ..........  3%  585K 7s
   150K .......... ...

CalledProcessError: Command 'b'url_base="https://download.inep.gov.br/enem/provas_e_gabaritos"\n\n\nfor ano in {2023..2023}; do\n    for dia in {1..12}; do\n        for caderno in {1..12}; do\n            # Usa a vari\xc3\xa1vel na URL corretamente\n            wget "${url_base}/${ano}_PV_impresso_D${dia}_CD${caderno}.pdf" -P ./provas --no-check-certificate\n        done\n    done\ndone\n'' returned non-zero exit status 8.

In [43]:
import fitz
import os

import numpy as np
import pandas as pd

import re

In [44]:
#Loader PDF to Text
class PdfToText():
  def __init__(self):
    pass

  def convert_from_file(self, filename):
    try:
      doc = self.load_pdf(filename)
    except:
      print( "Unable to open file." )
      return ""

    text = ""
    for page in doc:
      text += page.get_text()+"\n\n\n"

    doc.close()
    return text

  def load_pdf(self, filename):
    return fitz.open(filename)

class EnemExam(PdfToText):
  def __init__(self):
    pass

  def read_questions(self, filename):
    self.text = self.convert_from_file(filename)

    self.find_topics()
    self.find_questions()

  def find_topics(self):
    self._topics = ['matemática e suas tecnologias',
                    'linguagens, códigos',
                    'ciências humanas',
                    'ciências da natureza',
                    ]

    topic_poss = [ self.text.lower().rfind(topic) for topic in self._topics ]

    self._topics_loc = [ (topic,topic_pos) for topic, topic_pos in zip(self._topics, topic_poss) if topic_pos>0 ]
    self._topics_loc.sort( key=lambda t: t[1] )

  def find_questions(self):
    matchs = []
    for match in re.finditer('\n(questão) [0-9]+', self.text.lower()):
      matchs.append( match.start() )

    self.questions = []
    self.questions_topic = []
    self.questions_start = []
    for i in range( 1, len(matchs) ):
      self.questions.append( self.text[ matchs[i-1]:matchs[i] ] )
      self.questions_topic.append( self.define_question_topic(matchs[i-1]) )
      self.questions_start.append( matchs[i-1] )

  def define_question_topic(self, start):
    question_topic = 0 #UNDEFINED

    for topic, topic_start in self._topics_loc:
      if start > topic_start:
        question_topic = (self._topics.index(topic)+1)

    return question_topic

In [45]:
nome_arquivo_provas = os.listdir('./provas')
enem_exam = EnemExam()
enem_dfs = []

for nome_arquivo_prova in nome_arquivo_provas:
  enem_exam.read_questions('./provas/'+nome_arquivo_prova)
  enem_dfs.append(
      pd.DataFrame( data=dict( enunciado=enem_exam.questions, label=enem_exam.questions_topic )
    )
  )

In [49]:
from sklearn.model_selection import train_test_split

questions_df = pd.concat( enem_dfs )


train_df, test_df = train_test_split(questions_df, test_size=0.2, random_state=42)

print(len(train_df))
print(len(test_df))

803
201


In [51]:
train_df.to_json( 'data/train/enem_questoes.json', orient='records' )
test_df.to_json( 'data/test/enem_questoes.json', orient='records' )

In [52]:
from datasets import load_dataset

# Caminho para o diretorio
json_dir_path = "./data"

# Carregar o dataset a partir do diretorio
dataset = load_dataset("json", data_dir= json_dir_path)

# Exibir algumas amostras do dataset
print(dataset)

/opt/anaconda3/envs/llms/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 803 examples [00:00, 17086.86 examples/s]
Generating test split: 201 examples [00:00, 29390.10 examples/s]

DatasetDict({
    train: Dataset({
        features: ['enunciado', 'label'],
        num_rows: 803
    })
    test: Dataset({
        features: ['enunciado', 'label'],
        num_rows: 201
    })
})


In [54]:
import pandas as pd

dataset.set_format(type="pandas")
df = dataset["train"][:300]

df.head()

,enunciado,label
0,\nQUESTÃO 169\t\nO gráfico expõe alguns número...,1
1,\nQUESTÃO 120\t\nHá muito tempo são conhecidas...,4
2,"\nQUESTÃO 104\t\nOs solos amazônicos, ricos em...",4
3,\nQUESTÃO 58\t\nA diversão é o prolongamento d...,3
4,"\nQUESTÃO 15\t\nPassado muito tempo, resolvi t...",2
